# Tim's Data Analysis for Congressional District Mapping

The primary purpose of this notebook is for initial data exploration by Tim Romer. This notebook will feature http requests to api's as well as trying to explore the data set.

In [ ]:
# Meta-information for requesting data

CENSUS_API_KEY = '535963d4f87c22491889d73b8cc7889ee7d36465'
CURL_REQUEST = 'https://api.census.gov/data/2014/pep/natstprc.json?get=STNAME,POP&for=state:*&DATE_=7&key='+ CENSUS_API_KEY

To run this notebook, you will need to install an open source US Census Python wrapper. The link for install instructions is https://github.com/datamade/census

In [6]:
import census 
from us import states

c = census.Census("CENSUS_API_KEY")
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})

APIKeyError: ' <html>     <head>         <title>Invalid Key</title>     </head>     <body>         <p>             A valid <em>key</em> must be included with each data API request.             You included a key with this request, however, it is not valid.             Please check your key and try again.         </p>         <p>             If you do not have a key you my sign up for one <a href="key_signup.html">here</a>.         </p>     </body> </html>'